In [1]:
import datetime

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from Project.Database import Db

In [48]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)
year2_minutes, meta = Db.load_data(hourly=False, meta=True, year=2)

In [44]:
bath1 = meta.loc[(meta['Parameter'] == "Status_OnOff") & (meta['Measurement_Location'] == "Bath2") & (meta["Unnamed: 0"].str.contains('Status'))]['Unnamed: 0'].tolist()

In [46]:
fig = px.scatter(year1_minutes, x="Timestamp", y=bath1)
fig.show()

In [49]:
#Extract appliances from the meta data
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self["Unnamed: 0"].str.contains("Child" or "Prnt")) &
        (~self["Unnamed: 0"].str.contains("StatusLatentload"))
))

appliances_statusOnOff = meta.loc[appliance_condition]["Unnamed: 0"].tolist()
appliances_statusOnOff

['Load_StatusBA1Lights',
 'Load_StatusKitchenLightsA',
 'Load_StatusKitchenLightsB',
 'Load_StatusKitchenLightsC',
 'Load_StatusDRLights',
 'Load_StatusLRLights3',
 'Load_StatusEntryHallLights',
 'Load_StatusBR4Lights',
 'Load_StatusMBALights',
 'Load_StatusMBRLights1',
 'Load_StatusMBRLights2',
 'Load_StatusBA2Lights',
 'Load_StatusBR2Lights',
 'Load_StatusBR3Lights',
 'Load_StatusSensHeatPrntBDOWN',
 'Load_StatusSensHeatPrntAUP',
 'Load_StatusSensHeatPrntBUP',
 'Load_StatusLRLights1',
 'Load_StatusSensHeatPrntADOWN',
 'Load_StatusLRLights2',
 'Load_StatusPlugLoadBlender',
 'Load_StatusPlugLoadToasterOven',
 'Load_StatusApplianceRangeHood',
 'Load_StatusApplianceCooktop',
 'Load_StatusApplianceOven',
 'Load_StatusPlugLoadSlowCooker',
 'Load_StatusPlugLoadToaster',
 'Load_StatusApplianceDishwasher',
 'Load_StatusPlugLoadHandMixer',
 'Load_StatusPlugLoadCanOpener',
 'Load_StatusPlugLoadCoffeeMaker',
 'Load_StatusPlugLoadLRTV',
 'Load_StatusPlugLoadLRBlueRay',
 'Load_StatusPlugLoadVideoG

In [ ]:
#Combine minute data with the extracted appliances into a dataframe
minute_appliances_status = year1_minutes[["Timestamp"] + appliances_statusOnOff].copy()

In [51]:
#Replaces values to be binary 0 or 1.
for i in appliances_statusOnOff:
    minute_appliances_status.loc[minute_appliances_status[i] < 0, i] = 0
    minute_appliances_status.loc[minute_appliances_status[i] > 0, i] = 1
minute_appliances_status

,Timestamp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,Load_StatusEntryHallLights,Load_StatusBR4Lights,Load_StatusMBALights,...,Load_StatusPlugLoadVacuum,Load_StatusPlugLoadDesktopPCMonitor,Load_StatusPlugLoadHairDryerCurlIron,Load_StatusPlugLoadBR2Laptop,Load_StatusPlugLoadBR3Laptop,Load_StatusPlugLoadMBRTV,Load_StatusPlugLoadMBRBlueRay,Load_StatusPlugLoadFan,Load_StatusPlugLoadHeatingPad,Load_StatusPlugLoadIron
0,2013-07-01 00:01:14-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-07-01 00:02:14-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-07-01 00:03:14-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-07-01 00:04:14-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-07-01 00:05:14-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,2014-06-30 23:29:22-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
518788,2014-06-30 23:30:22-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
518789,2014-06-30 23:31:22-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
518790,2014-06-30 23:59:22-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Her udtrækkes start og slut tidspunkt for hvert appliance når dette er tændt og gemmes i DF
#eks: https://stackoverflow.com/questions/36864690/iterate-through-a-dataframe-by-index

for index in range(minute_appliances_status.shapes(1)):
    for i, row in minute_appliances_status.iterrows():
        index = i
        Load_StatusBA1Lights = row["Load_StatusBA1Lights"]